In [1]:
from pathlib import Path
import os
from urllib.parse import urljoin

from astropy.io import fits
from astropy.time import Time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import padre_meddea
import padre_meddea.calibration.calibration as calib
from padre_meddea.io import file_tools
from padre_meddea.io import fits_tools
from padre_meddea.util.util import (
    parse_science_filename,
    calc_time,
)


# Time Correction Protocol

Based on the signature of `padre_meddea.util.util.calc_time`:
- `pkt_time_s`: u.s (Seconds)
- `pkt_time_clk`: u.microseconds (Microseconds)
- `ph_clk`: u.microseconds (Microseconds)

Within each HDU, the following Table Columns are used for timing:
- Photon Files: Variable Cadence files so we're not correcting these.
- Spectrum Files: 
    - PKT HDU uses `pkt_time_s=pkttimes` and `pkt_time_clk=pktclock`
        - Correct using `table.iloc[wrong_time][<what_column>] = table.iloc[nearest_correct_time][<what_column>] +/- <what cadence>` 
    - SPEC HDU uses the PKT HDU for timing information.
        - Correct using `table.iloc[wrong_time][<what_column>] = table.iloc[nearest_correct_time][<what_column>] +/- <what cadence>` 
- Housekeeping Files:
    - HK HDU uses `pkt_time_s=timestamp`
        - Correct using `table.iloc[wrong_time][<what_column>] = table.iloc[nearest_correct_time][<what_column>] +/- <what cadence>` 
    - READ HDU uses `pkt_time_s=pkttimes` and `pkt_time_clk=pktclock`
        - Correct using `table.iloc[wrong_time][<what_column>] = table.iloc[nearest_correct_time][<what_column>] +/- <what cadence>`


# Download the RAW Files

In [2]:
def download_from_url(base_url, experiment_path, recurse=True, file_extension='.fits'):
    """
    Recursively download files from a URL, preserving directory structure.
    
    Args:
        base_url (str): The URL to download files from
        experiment_path (Path): The local path to save files to
        recurse (bool): Whether to recursively download from subdirectories
        file_extension (str): File extension to filter by (e.g., '.fits')
    """
    try:
        print(f"Accessing {base_url}")
        response = requests.get(base_url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        parent_folder = BeautifulSoup(response.text, 'html.parser')
        
        # Create the local directory if it doesn't exist
        os.makedirs(experiment_path, exist_ok=True)
        
        for link in parent_folder.find_all('a'):
            href = link.get('href')
            
            # Skip parent directory links and query parameters
            if not href or href.startswith('?') or href.startswith('/') or href == '../':
                continue
                
            # Create full URL for the link
            full_url = urljoin(base_url, href)
            
            # If it's a directory and we're recursing
            if href.endswith('/') and recurse:
                # Remove trailing slash for local directory name
                dir_name = href[:-1]
                local_dir = experiment_path / dir_name
                
                # Recursively download from this directory
                download_from_url(full_url, local_dir, recurse, file_extension)
            
            # If it's a file with the desired extension
            elif href.lower().endswith(file_extension.lower()):
                print(f"Downloading {full_url}")
                local_path = experiment_path / href
                
                # Create parent directories if they don't exist
                os.makedirs(local_path.parent, exist_ok=True)
                
                # Download the file
                with open(local_path, 'wb') as file:
                    file_response = requests.get(full_url)
                    file_response.raise_for_status()
                    file.write(file_response.content)
                    
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {base_url}: {e}")
    except Exception as e:
        print(f"Error processing {base_url}: {e}")

In [3]:
### L1 Files
testfile_path = Path.cwd() / 'test_files'
if not testfile_path.exists():
    testfile_path.mkdir(parents=True, exist_ok=True)
base_url = 'https://umbra.nascom.nasa.gov/padre/padre-meddea/raw/2025/05/04/'
download_from_url(base_url, testfile_path, recurse=True, file_extension='.DAT')

Accessing https://umbra.nascom.nasa.gov/padre/padre-meddea/raw/2025/05/04/


# Process the RAW Files to L0

In [4]:
cwd = Path.cwd()
os.chdir(testfile_path)
for file in testfile_path.rglob('*.DAT'):
    try:
        processed_files = calib.process_file(file, overwrite=True)
        print(f"Processed {file.name} into:")
        for processed_file in processed_files:
            print(f"  {processed_file}")
    except Exception as e:
        print(f"Error processing {file.name}: {e}")
# Change back to the original working directory
os.chdir(cwd)

2025-07-11 12:23:06 - swxsoc - INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDA0_250504095122.DAT.


INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDA0_250504095122.DAT. [padre_meddea.calibration.calibration]


2025-07-11 12:23:06 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDA0_250504095122.DAT : UserWarning: Sequence count are out of order.
2025-07-11 12:23:06 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDA0_250504095122.DAT : ChecksumWarning: Packet 548 has a checksum error.
2025-07-11 12:23:06 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDA0_250504095122.DAT : ChecksumWarning: Packet 549 has a checksum error.
2025-07-11 12:23:06 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDA0_250504095122.DAT 

INFO: PADREMDA0_250504095122.DAT: Found photon data [padre_meddea.spectrum.raw]


/opt/homebrew/Caskroom/miniforge/base/envs/swxsoc/lib/python3.12/site-packages/ccsdspy/packet_types.py:364: UserWarning: Sequence count are out of order.
  warnings.warn("Sequence count are out of order.", UserWarning)
2025-07-11 12:23:07 - swxsoc - INFO: PADREMDA0_250504095122.DAT: Found baseline measurements in photon data.


...

Processed PADREMDA0_250504085936.DAT into:
  padre_meddea_l0test_photon_20000101T000000_v0.1.0.fits
INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDA2_250504081536.DAT. [padre_meddea.calibration.calibration]
INFO: PADREMDA2_250504081536.DAT: Found spectrum data [padre_meddea.spectrum.raw]


/opt/homebrew/Caskroom/miniforge/base/envs/swxsoc/lib/python3.12/site-packages/ccsdspy/packet_types.py:361: UserWarning: Missing packets found [869, 871, 1117].
  warnings.warn(f"Missing packets found {missing_elements}.", UserWarning)
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-07-11 12:26:06 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-07-11 12:26:06 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-07-11 12:26:06 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-07-11 12:26:06 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Ti

Processed PADREMDA2_250504081536.DAT into:
  padre_meddea_l0test_spectrum_20250504T081521_v0.1.0.fits
INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/meddea_notebooks/experiments/2025-07-10-time-interpolation/test_files/PADREMDU8_250504153121.DAT. [padre_meddea.calibration.calibration]
INFO: PADREMDU8_250504153121.DAT: Found housekeeping data [padre_meddea.housekeeping.housekeeping]


2025-07-11 12:26:11 - swxsoc - WARNING: Version 3.0.1.dev42+gb90591f not found in Tags for https://github.com/IHDE-Alliance/solarnet_metadata.git. Using HEAD.
2025-07-11 12:26:12 - astropy - WARNING: VerifyWarning: Card is too long, comment will be truncated.


Processed PADREMDU8_250504153121.DAT into:
  padre_meddea_l0test_housekeeping_20250504T153118_v0.1.0.fits


In [5]:
file_meta = []
for file in testfile_path.glob("*.fits"):
    print(f"Processing file: {file.name}")
    # Open the FITS file
    hdul_dict = fits_tools._init_hdul_structure(file)
    for i in hdul_dict:
        if hdul_dict[i]["data"] is None or len(hdul_dict[i]["data"]) == 0:
            continue
        # Get Time Range for HDU
        times = fits_tools.get_hdu_data_times(hdul_dict, hdul_dict[i]["name"])
        meta = parse_science_filename(file.name)
        meta["start_time"] = times[0]
        meta["end_time"] = times[-1]
        meta["hdu_name"] = hdul_dict[i]["name"]
        meta["length"] = len(times)
        if len (times) > 1:
            meta["cadence (s)"] = times[-1].mjd - times[-2].mjd
        else:
            meta["cadence (s)"] = 0
        file_meta.append(meta)

Processing file: padre_meddea_l0test_photon_20250504T153114_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T101642_v0.1.0.fits
Processing file: padre_meddea_l0test_spectrum_20250504T103811_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T084605_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T144630_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T073749_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20000101T000000_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T133757_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T083234_v0.1.0.fits
Processing file: padre_meddea_l0test_spectrum_20250504T114921_v0.1.0.fits
Processing file: padre_meddea_l0test_housekeeping_20250504T153118_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T080330_v0.1.0.fits
Processing file: padre_meddea_l0test_photon_20250504T124415_v0.1.0.fits
Processing file: padre_meddea_l0test_spectrum_20250504

In [6]:
df = pd.DataFrame(file_meta).sort_values(by=["start_time", "end_time"])
df.head()

,instrument,mode,test,time,level,version,descriptor,start_time,end_time,hdu_name,length,cadence (s)
63,meddea,None,True,2025-05-04T05:51:38.000,l0,0.1.0,housekeeping,2025-05-04 05:51:38.000,2025-05-04 15:31:08.000,HK,3478,1.157407e-04
64,meddea,None,True,2025-05-04T05:51:38.000,l0,0.1.0,housekeeping,2025-05-04 05:53:06.206,2025-05-04 05:53:06.206,READ,1,0.000000e+00
34,meddea,None,True,2025-05-04T05:53:11.000,l0,0.1.0,photon,2025-05-04 05:53:11.353,2025-05-04 07:37:49.299,PKT,16669,3.207286e-06
33,meddea,None,True,2025-05-04T05:53:11.000,l0,0.1.0,photon,2025-05-04 05:53:11.353,2025-05-04 07:37:49.472,SCI,1264891,1.775334e-09
29,meddea,None,True,2025-05-04T07:04:11.000,l0,0.1.0,spectrum,2025-05-04 07:04:11.349,2025-05-04 08:15:11.363,SPEC,426,1.157411e-04


In [7]:
# Filter the DataFrame for records with the specific start time
bad_time_df = df[df["start_time"] == "2000-01-01 00:00:00.000"]

# Group by descriptor and hdu_name and count occurrences
bad_time_counts = bad_time_df.groupby(["descriptor", "hdu_name"])["start_time"].value_counts()

# Display the results
bad_time_counts

Series([], Name: count, dtype: int64)

**Original Result**

We can see 
- Housekeeping does not have any data with incorrect timestamps. (no `2000-01-01` timestamps)
- Photon data looks like it has one files with a `2000-01-01` timestamp.
- Spectrum data has multiple files with `2000-01-01` timestamps. Looks like 7 files

```
descriptor  hdu_name  start_time             
photon      PKT       2000-01-01 00:00:00.000    1
            SCI       2000-01-01 00:00:00.000    1
spectrum    PKT       2000-01-01 00:00:00.000    7
            SPEC      2000-01-01 00:00:00.000    7
Name: count, dtype: int64
```